<a href="https://colab.research.google.com/github/atick-faisal/TAVI/blob/main/src/colab/MultiViewUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Info

In [ ]:
from psutil import virtual_memory

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Mon Jan 23 19:31:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download Dataset

In [ ]:
!pip install -U --no-cache-dir gdown --pre
!pip install image-similarity-measures > /dev/null

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown "12CxxS2w7mprCnVX9Y_Kiti6vEPbTQW2l"
!gdown "1CN-7DANpVbSzkxmGOj_zGX6QC0q1HHDM"
!unzip -o "CurvToTAWSSJet.zip" > /dev/null


Downloading...
From: https://drive.google.com/uc?id=12CxxS2w7mprCnVX9Y_Kiti6vEPbTQW2l
To: /content/UNet_2DCNN.py
100% 58.6k/58.6k [00:00<00:00, 74.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GfS5qapaM5Ci7ykHQW_4B-kWqk_tmvSd
To: /content/TAWSS2ECAP_v1.zip
100% 1.06G/1.06G [00:06<00:00, 165MB/s]


# Imports

In [ ]:
import os
import cv2
import random
import datetime
import matplotlib
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from numpy.typing import NDArray
from UNet_2DCNN import UNet as UNetSakib
# from image_similarity_measures.quality_metrics import *

matplotlib.use('Agg')
plt.rcParams["font.size"] = 16


# Config

In [ ]:
DATASET         = "CurvToTAWSS"
MODEL_NAME      = "MultiViewUNet"
DATASET_PATH    = "/content/Images/"
TRAIN_DIR       = "Train/"
TEST_DIR        = "Test/"
INPUT_DIR       = "Input/"
TARGET_DIR      = "Target/"
MODEL_PATH      = "/content/drive/MyDrive/Research/TAVI/Models/"
PRED_PATH       = "/content/drive/MyDrive/Research/TAVI/Predictions/"

IMG_SIZE        = 256
BATCH_SIZE      = 16
BUFFER_SIZE     = 1000
VAL_SPLIT       = 0.2
LEARNING_RATE   = 0.001
N_EPOCHS        = 300
PATIENCE        = 30

EXP_NAME        = f"{MODEL_NAME}_I{IMG_SIZE}_B{BATCH_SIZE}_LR{LEARNING_RATE}_{DATASET}"


# Architecture

In [ ]:
class UNet:
    def __init__(
        self,
        img_size: int,
        n_channels: int = 3,
        width: int = 32,
        depth: int = 4,
        kernel_size: int = 3
    ):
        self.img_size = img_size
        self.n_channels = n_channels
        self.width = width
        self.depth = depth
        self.kernel_size = kernel_size

    @staticmethod
    def conv(
        x: tf.Tensor,
        filters: int,
        kernel_size: int
    ) -> tf.Tensor:
        for i in range(2):
            x = tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=kernel_size,
                strides=1,
                padding="same",
                data_format="channels_last",
                dilation_rate=1,
                groups=1,
                activation=None,
                use_bias=True,
                kernel_initializer="glorot_uniform",
                bias_initializer="zeros",
            )(x)

            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.Activation("relu")(x)

        return x

    @staticmethod
    def deconv(x: tf.Tensor, filters: int) -> tf.Tensor:
        x = tf.keras.layers.Conv2DTranspose(
            filters=filters,
            kernel_size=2,
            strides=2,
            padding="same",
            output_padding=None,
            data_format=None,
            dilation_rate=1,
            activation=None,
            use_bias=True,
            kernel_initializer="glorot_uniform",
            bias_initializer="zeros",
        )(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)

        return x

    @staticmethod
    def output(x: tf.Tensor) -> tf.Tensor:
        return tf.keras.layers.Conv2D(3, (1, 1), activation="sigmoid")(x)

    @staticmethod
    def pool(x: tf.Tensor, pool_size: int = 2) -> tf.Tensor:
        return tf.keras.layers.MaxPool2D(pool_size)(x)

    @staticmethod
    def dropout(x: tf.Tensor, amount: float = 0.5) -> tf.Tensor:
        return tf.keras.layers.Dropout(amount)(x)

    def __call__(self) -> tf.keras.Model:
        inputs = tf.keras.layers.Input(
            shape=(self.img_size, self.img_size, self.n_channels)
        )

        # scaled = tf.keras.layers.Rescaling(1./255.0, offset=0)(inputs)

        # ------------------ Downsampling ---------------------
        downsample_layers = []
        downsample_layers.append(
            self.conv(
                x=inputs,
                filters=self.width,
                kernel_size=self.kernel_size
            )
        )
        for i in range(1, self.depth):
            dropout_amount = 0.2 if i == 1 else 0.5
            filters = int((2 ** i) * self.width)
            downsample_layers.append(
                self.dropout(
                    self.pool(
                        self.conv(
                            x=downsample_layers[i - 1],
                            filters=filters,
                            kernel_size=self.kernel_size
                        )
                    ),
                    amount=dropout_amount
                )
            )

        # ------------------- Features --------------------
        n_features = int((2 ** self.depth) * self.width)
        self.features = self.pool(
            self.conv(
                x=downsample_layers[-1],
                filters=n_features,
                kernel_size=self.kernel_size
            )
        )

        # ------------------- Upsampling --------------------
        upsample_layers = []
        upsample_layers.append(self.features)
        for i in range(1, self.depth + 1):
            filters = int((2 ** (self.depth - i)) * self.width)
            upsample_layers.append(
                self.conv(
                    x=self.dropout(
                        tf.keras.layers.concatenate([
                            downsample_layers[self.depth - i],
                            self.deconv(
                                x=upsample_layers[i - 1],
                                filters=filters
                            )
                        ])
                    ),
                    filters=filters,
                    kernel_size=self.kernel_size
                )
            )

        # ---------------------- Output -----------------------
        outputs = self.output(upsample_layers[-1])

        return tf.keras.Model(inputs, outputs)


# Loss Functions

In [ ]:
def attention_mse(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.square(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def attention_mae(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.abs(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


# Utils

In [ ]:
def load_data_from_dir(path: str) -> tf.data.Dataset:
    return tf.keras.utils.image_dataset_from_directory(
        directory=path,
        labels=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=True,
        seed=42,
        # validation_split=VAL_SPLIT,
        # subset=subset,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )


# Dataloaders

In [ ]:
trainX = load_data_from_dir(os.path.join(DATASET_PATH, TRAIN_DIR, INPUT_DIR))
trainY = load_data_from_dir(os.path.join(DATASET_PATH, TRAIN_DIR, TARGET_DIR))
testX = load_data_from_dir(os.path.join(DATASET_PATH, TEST_DIR, INPUT_DIR))
testY = load_data_from_dir(os.path.join(DATASET_PATH, TEST_DIR, TARGET_DIR))

train_ds = tf.data.Dataset.zip((trainX, trainY))
test_ds = tf.data.Dataset.zip((testX, testY))

print(train_ds.element_spec)
print(test_ds.element_spec)


Found 8208 files belonging to 1 classes.
Found 8208 files belonging to 1 classes.
Found 312 files belonging to 1 classes.
Found 312 files belonging to 1 classes.
(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))
(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))


# Normalization

In [ ]:
# def normalize(input_image, output_image):
#     input_image = tf.cast(input_image, tf.float32) / 255.0
#     output_image = tf.cast(input_image, tf.float32) / 255.0
#     return input_image, output_image

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), normalization_layer(y)))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), normalization_layer(y)))

# Augmentation

In [ ]:
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = tf.keras.layers.RandomZoom((-0.1, -0.7), seed=seed)
        self.augment_labels = tf.keras.layers.RandomZoom((-0.1, -0.7), seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels

# Optimization

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_batches = (
    train_ds
    .cache()
    # .shuffle(BUFFER_SIZE)
    # .batch(BATCH_SIZE)
    # .repeat()
    # .map(Augment())
    .prefetch(buffer_size=AUTOTUNE)
)

test_batches = (test_ds)


# Training Config

In [ ]:
model_path = os.path.join(MODEL_PATH, EXP_NAME)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=PATIENCE,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        model_path,
        monitor = 'val_loss',
        verbose = 1,
        save_best_only = True,
        save_weights_only = True
    )
]

optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE
)

model = UNet(IMG_SIZE)()
# model = MobileUNet(IMG_SIZE)()
# model = UNetSakib(
#     length=IMG_SIZE, 
#     width=IMG_SIZE, 
#     model_depth=4, 
#     num_channel=1, 
#     model_width=32, 
#     kernel_size=3, 
#     problem_type='Regression',
#     output_nums=1, 
#     ds=0, 
#     ae=0, 
#     ag=0, 
#     lstm=0, 
#     alpha=1, 
#     feature_number=1024, 
#     is_transconv=True
# ).FSC_Net()

model.compile(
    loss=attention_mse,
    optimizer=optimizer,
    metrics=[attention_mae]
)

# try:
#     model.load_weights(model_path)
# except:
#     print("Checkpoint not found")
#     pass


# Training

In [ ]:
history = model.fit(
    train_batches,
    validation_data=test_batches,
    epochs=N_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/300
513/513 [==============================] - ETA: 0s - loss: 0.0385 - attention_mae: 0.1331
Epoch 1: val_loss improved from inf to 0.03887, saving model to /content/drive/MyDrive/Research/TAVI/Models/MultiViewUNet_I256_B16_LR0.001_TAWSS2ECAP
513/513 [==============================] - 311s 568ms/step - loss: 0.0385 - attention_mae: 0.1331 - val_loss: 0.0389 - val_attention_mae: 0.1435
Epoch 2/300
513/513 [==============================] - ETA: 0s - loss: 0.0320 - attention_mae: 0.1172
Epoch 2: val_loss improved from 0.03887 to 0.03146, saving model to /content/drive/MyDrive/Research/TAVI/Models/MultiViewUNet_I256_B16_LR0.001_TAWSS2ECAP
513/513 [==============================] - 288s 561ms/step - loss: 0.0320 - attention_mae: 0.1172 - val_loss: 0.0315 - val_attention_mae: 0.1099
Epoch 3/300
513/513 [==============================] - ETA: 0s - loss: 0.0302 - attention_mae: 0.1124
Epoch 3: val_loss did not improve from 0.03146
513/513 [==============================] - 287s 560ms

# Save Model

In [ ]:
timestamp = datetime.datetime.now().strftime('%b-%d-%I:%M%p')
if not os.path.exists(model_path):
    os.makedirs(model_path)

model.save(os.path.join(model_path, timestamp))

# Save Predictions

In [ ]:
test_ds_unbatched = test_batches.unbatch()

pred_path = os.path.join(PRED_PATH, EXP_NAME, timestamp)
if not os.path.exists(pred_path):
    os.makedirs(pred_path)

metrics = pd.DataFrame()

for idx, (input, target) in enumerate(tqdm(test_ds_unbatched)):

    # if not idx % 10 == 0:
    #     continue

    target = tf.squeeze(target).numpy()
    prediction = tf.squeeze(
        model.predict(
            tf.expand_dims(input, axis=0)
            )
        ).numpy()
    prediction[target == 1.0] = 1.0

    # _metrics = pd.DataFrame([[
    #         rmse(target, prediction),
    #         psnr(target, prediction),
    #         ssim(target, prediction),
    #         fsim(target, prediction),
    #         issm(target, prediction),
    #         sre(target, prediction),
    #         sam(target, prediction),
    #         uiq(target, prediction),
    #     ]],
    #     columns=["RMSE", "PSNR", "SSIM", "FSIM", "ISSM", "SRE", "SAM", "UIQ"]
    # )

    # metrics = pd.concat([metrics, _metrics], axis=0)

    plt.figure(figsize=(7, 7))
    plt.imshow(target)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}_T.png"))
    plt.close()

    plt.figure(figsize=(7, 7))
    plt.imshow(prediction)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}_P.png"))
    plt.close()


    # plt.figure(figsize=(12, 4))
    # plt.subplot(1, 3, 1)
    # plt.imshow(input)
    # plt.axis("off")
    # plt.title("INPUT")
    # plt.subplot(1, 3, 2)
    # plt.imshow(target)
    # plt.axis("off")
    # plt.title("TARGET")
    # plt.subplot(1, 3, 3)
    # plt.imshow(prediction)
    # plt.axis("off")
    # plt.title("PREDICTION")
    # plt.tight_layout()
    # plt.savefig(os.path.join(pred_path, f"{idx}.png"))
    # plt.close()  


0it [00:00, ?it/s]

1/1 [==============================] - 2s 2s/step


1it [00:02,  2.90s/it]

1/1 [==============================] - 0s 18ms/step


2it [00:03,  1.37s/it]

...

310it [01:47,  3.02it/s]

1/1 [==============================] - 0s 19ms/step


311it [01:47,  2.93it/s]

1/1 [==============================] - 0s 19ms/step


312it [02:21,  2.20it/s]


In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 6))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Attention MAE')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.tight_layout()
plt.savefig(os.path.join(model_path, timestamp + ".png"))
plt.close()

In [ ]:
model.evaluate(test_batches)

20/20 [==============================] - 4s 150ms/step - loss: 0.0156 - attention_mae: 0.0603


[0.015646839514374733, 0.06028332561254501]